<a href="https://colab.research.google.com/github/nathanzilgo/Data-Science-Machine-Learning/blob/master/LDA_tweets_por_autor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usar LDA para agrupar autores de tweets em tópicos

---


> **TODOS**:
* Incrementar as stopwords para os tweets
* Testar valores diferentes no plate notation
* Implementar uma visualização gráfica

In [ ]:
import pandas
import numpy as np
import altair
import zipfile 

from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

from google.colab import drive



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Monta a pasta do drive com o zip dos tweets. Só precisa ser executado uma vez na criação do notebook.

In [ ]:
drive.mount('/content/drive')

zf = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/tweets_parlamentares_new.zip", 'r')
tweets_csv = zf.extractall()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
raw = pandas.read_csv('./tweets_parlamentares_new.csv', nrows=100000)

tweets = raw

print('Total de tweets: {0}'.format(tweets.shape[0]))
print()
print('Por parlamentar: ')
print()
tweets['nome_eleitoral'].value_counts()

test = tweets.loc[tweets['nome_eleitoral'] == 'ALEXANDRE PADILHA']
test['nome_eleitoral'].value_counts()

Total de tweets: 100000

Por parlamentar: 



ALEXANDRE PADILHA    5410
Name: nome_eleitoral, dtype: int64

# Pre-processamento
> Remover algumas stopwords

In [ ]:
# Remover stopwords em portugues
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'bb', 
                    'bbm', 'abm', 'cbm', 
                    'dbm', 'dos', 
                    'ltda', 'editora', 'nan', 'https',
                    '__twitter_impression', '__twitter_impression true', 'http', 'http bit'])

clean_tweets = list()

for w in range(len(test.text)):
  tw = str(test['text'].iloc[w])

  # remove special characters and digits
  tw  = re.sub("(\\d|\\W)+|\w*\d\w*"," ", tw )
  tw = ' '.join(s for s in tw.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
  clean_tweets.append(tw)

clean_tweets[20:50]


['mortos casos confirmados Enquanto isso que Bolsonaro estava fazendo Sobrevoando helicoptero uma manifestação Brasília',
 'dias sem ministro Saúde',
 'Hoje dia mundial Sem Tabaco entrevista concedida quando era Ministro Saúde falo pouco sobre legado luta contra tabagismo Entrevista completa https www youtube com watch OlKzdotsCY',
 'Depois que torcidas rivais estiveram juntas hoje todo Brasil lideranças políticas têm que amanhã baixarem suas bandeiras próprias erguerem juntos bandeirão pela DEMOCRACIA',
 'Companheiros companheiras dos partidos progressistas vamos aproveitar que torcidas Corinthians Palmeiras uniram contra facismo nos unirmos vez Tenho certeza que concordam com isso FlavioDino Haddad_Fernando ManuelaDavila GuilhermeBoulos gleisi',
 'taco baseball uma bandeira dos Estados Unidos encoberta pelos braços força ordem diz muito sobre onda conservadora que assola Brasil favor democracia sempre democracia fascismonuncamais',
 'ato contra Bolsonaro manifestante arrastada detida

### LDA funciona baseado em frequências de palavras, então usaremos TFs, e não TF-IDFs para os textos dos tweets.

* **max_df**: ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts.

* **min_df**: ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts.

Neste caso, tive resultados interessantes com mindf = 1, max_df = 0.4, max_features= 10000 e ngram_range(1,2)

In [ ]:
# Define o "comportamento" do vetorizador
tf_vectorizer = CountVectorizer(
    min_df = 10,
    max_df = 0.8,
    max_features = None,
    stop_words = stop_words,
    ngram_range=(1,2)
)

# Aplica o vectorizer para os textos dos tweets
vec_text = tf_vectorizer.fit_transform(clean_tweets)

# Lista de palavras
words = tf_vectorizer.get_feature_names()

# Formato do vetor:
print("Formato do vetor: ", vec_text.shape)
# Quantidade de palavras (features):
print("Quantidade de features: ", len(words))
# visualizar o vetor de features (palavras):
print("Vetor de features: \n", vec_text[0:10])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bit', 'true'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Formato do vetor:  (5410, 1537)
Quantidade de features:  1537
Vetor de features: 
   (0, 885)	1
  (0, 318)	3
  (0, 1053)	1
  (0, 1090)	1
  (0, 1225)	1
  (0, 1455)	1
  (0, 91)	1
  (0, 1155)	1
  (0, 1517)	1
  (0, 995)	1
  (0, 1224)	1
  (1, 1061)	1
  (1, 668)	1
  (1, 294)	1
  (1, 1445)	1
  (1, 1307)	1
  (1, 1086)	1
  (1, 1433)	1
  (3, 630)	1
  (3, 1197)	1
  (3, 146)	1
  (3, 789)	1
  (3, 1491)	1
  (3, 731)	1
  (4, 1155)	1
  :	:
  (6, 116)	1
  (6, 1183)	1
  (6, 1194)	1
  (6, 353)	1
  (7, 318)	1
  (7, 886)	1
  (7, 214)	1
  (7, 1122)	1
  (7, 1239)	1
  (7, 963)	1
  (7, 1161)	1
  (7, 587)	1
  (7, 1124)	1
  (8, 1284)	1
  (8, 426)	1
  (8, 873)	1
  (8, 875)	1
  (9, 789)	1
  (9, 645)	1
  (9, 380)	1
  (9, 478)	1
  (9, 852)	1
  (9, 378)	1
  (9, 1027)	1
  (9, 407)	1


In [ ]:
words[0:10]

['aberta',
 'aberto',
 'abrahamweint',
 'abraço',
 'abril',
 'absurda',
 'absurdo',
 'acaba',
 'acabar',
 'acabou']

## Encontrando tópicos

O resultado terá 

*   uma matriz que descreve a relação entre palavras e tópicos
*   uma matriz que descreve a relação entre documentos e tópicos

Existe uma outra implementação de LDA popular em python chamada Gensim.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Utilitários de visualização de dados do Prof Nazareno
def print_top_words(model, feature_names, n_top_words):

    for topic_idx, topic in enumerate(model.components_):

        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        message = ", ".join([feature_names[i]
                              for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

    print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):

    for topic_idx, topic in enumerate(H):

        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        
        for d in top_d_idx: 
          doc_data = tweets[['nome_eleitoral', 'status_url']].iloc[d]
          print('{} - {} : \t{:.2f}'.format(doc_data[1], doc_data[0], W[d, topic_idx]))

## **Agrupamento (Dúvidas)**

### Atributos do Plate Notation:
* **doc_topic_prior <float, optional (default=None)>**: 
Prior of document topic distribution theta

* **topic_word_prior <float, optional (default=None)>**: Prior of topic word distribution beta


In [ ]:
lda = LatentDirichletAllocation(n_components=5, # número de tópicos
                                # doc_topic_prior=5,
                                # topic_word_prior=0.6, # Prior of topic word distribution beta
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0) # obter os mesmos resultados aleatórios

t0 = time() 

lda.fit(vec_text) # Learn model from vectorized text

doc_topic_matrix = lda.transform(vec_text) # Matriz de relação docs x topics

print("done in %0.3fs." % (time() - t0))

done in 7.432s.


In [ ]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(5410, 5)
Matriz tópicos-termos:(5, 1537)


In [ ]:
# TODO: Tirar dúvida de como os tópicos estão sendo agrupados
display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               test,
               15, 
               10)


--
Topic #1: 
PRESIDENTE, PESSOAS, DIA, ANOS, POVO, LULA, DIREITOS, MUNDO, BOLSONARO, MILHÕES, BIT, MULHERES, NUNCA, DESTRUIÇÃO, MANDATO
https://twitter.com/adrianasounovo/status/1242186643148333059 - ADRIANA VENTURA : 	0.95
https://twitter.com/adrianasounovo/status/1186713542428430341 - ADRIANA VENTURA : 	0.95
https://twitter.com/adrianasounovo/status/1196056967111938048 - ADRIANA VENTURA : 	0.94
https://twitter.com/adrianasounovo/status/1245836428057235457 - ADRIANA VENTURA : 	0.94
https://twitter.com/adrianasounovo/status/1184133735773593601 - ADRIANA VENTURA : 	0.93
https://twitter.com/adrianasounovo/status/1260947079976955905 - ADRIANA VENTURA : 	0.93
https://twitter.com/AdrianoBaldy/status/1212841481154764803 - ADRIANO DO BALDY : 	0.93
https://twitter.com/adrianasounovo/status/1242798623995441157 - ADRIANA VENTURA : 	0.92
https://twitter.com/adrianasounovo/status/1093239887128678400 - ADRIANA VENTURA : 	0.92
https://twitter.com/adrianasounovo/status/1263503460126818304 - ADRIANA